In [5]:
import os
import pandas as pd
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from tagger.core.mongo.models.order import Order
from pytz import timezone
from datetime import datetime
from tagger.models.inventory import Inventory, InventoryStatus


TZ = timezone("Asia/Seoul")
# TARGETS = [
#     "61812c7d26685ff051754cee",
# ]

to_be_shipped = Inventory.objects.filter(status=InventoryStatus.SHIPPING_PENDING).all()
TARGETS = [x.out_order_id for x in to_be_shipped]

CJ_COLUMNS = [
    "주문번호", "주문일", "수령자명", "수령자 우편번호", "수령자 연락처", "수령자주소", "상품명", "옵션명", "수량", "배송메모", "박스타입"
]



def get_cj_row(order):
    payment = order.get_payment()
    return [
        order.code.replace("ORD-", ""),
        order.created.astimezone(TZ).date().isoformat().replace("-", "/"),
        payment.buyername,
        payment.buyerpostcode,
        f"{payment.buyermobile[:3]}-{payment.buyermobile[3:7]}-{payment.buyermobile[7:11]}",
        payment.buyeraddress,
        payment.name,
        ",".join([f"{item.size}/{item.quantity}EA" for item in order.orders]),
        1,
        order.memo,
        "소"
    ]

def get_cu_row(order):
    payment = order.get_payment()
    return [
        payment.buyername,
        payment.buyerpostcode,
        payment.buyeraddress,
        payment.buyeraddress,
        f"{payment.buyermobile[:3]}-{payment.buyermobile[3:7]}-{payment.buyermobile[7:11]}",
        "",
        order.memo,
        "선불"
    ]

def copy_cu():
    orders = Order.objects(id__in=TARGETS)
    df = pd.DataFrame([get_cu_row(order) for order in orders])
    df[1] = df[1].apply(lambda x: x.zfill(5))
    df.to_clipboard()

def make_cj():
    orders = Order.objects(id__in=TARGETS)
    df = pd.DataFrame([get_cj_row(order) for order in orders], columns=CJ_COLUMNS)
    df["수령자 우편번호"] = df["수령자 우편번호"].apply(lambda x: x.zfill(5))
    with open(f"../Downloads/cj_upload_{datetime.now().date().isoformat()}.xlsx", "wb") as file:
        df.to_excel(file, index=None)


In [6]:
make_cj()

FileNotFoundError: [Errno 2] No such file or directory: '~/cj_upload.xls'

'2021-10-17'